In [1]:
%%bash
pip install -qq /kaggle/input/wheels/lightning-2.4.0-py3-none-any.whl;

pip install --no-deps -qq /kaggle/input/wheels/omegaconf-2.3.0-py3-none-any.whl;
pip install --no-deps -qq /kaggle/input/wheels/einops-0.7.0-py3-none-any.whl;

cp -r /kaggle/input/wheels/antlr4-python3-runtime-4.9.3/antlr4-python3-runtime-4.9.3 /kaggle/working/;
cd /kaggle/working/antlr4-python3-runtime-4.9.3;
pip install . --user;

pip install --no-deps -qq /kaggle/input/wheels/pytorch_tabnet-4.1.0-py3-none-any.whl;
pip install --no-deps -qq /kaggle/input/wheels/pytorch_tabular-1.1.1-py2.py3-none-any.whl;

Processing /kaggle/working/antlr4-python3-runtime-4.9.3
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=429364d264e5f54db761fc543a29bd4834d6070ed89f0e25f27f91a7adffd229
  Stored in directory: /root/.cache/pip/wheels/08/67/2a/c8a92440bcfa6a48a4e0b84bd0f5ffbf135907bacb6ee4ae0e
Successfully built antlr4-python3-runtime


In [2]:
import numpy as np
import pandas as pd
from typing import Optional, List, Callable
import polars
import os
import pickle

import torch 
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from torch.optim.lr_scheduler import OneCycleLR

import catboost as cb
print("cb.__version__:", cb.__version__)

import lightgbm as lgb
print("Lightgbm version:", lgb.__version__)

import xgboost as xgb
print("xgb.__version__:", xgb.__version__)

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint

from pytorch_tabular.models.common.layers import GatedFeatureLearningUnit
from pytorch_tabular.models.common.layers.activations import t_softmax

print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Lightning version: {pl.__version__}")

import sys
sys.path.append("/kaggle/input/mcts-artifacts")
from preproc import process_test_data
import kaggle_evaluation.mcts_inference_server

cb.__version__: 1.2.7
Lightgbm version: 4.2.0
xgb.__version__: 2.0.3
PyTorch version: 2.4.0
PyTorch Lightning version: 2.4.0


***
### 1dcnn

In [3]:
# Specify the path where you want to save the serialized function

nn_1dcnn_artifacts_path = '/kaggle/input/mcts-artifacts/nn-1dcnn_predict_uni80.pt'
# nn_1dcnn_artifacts_path = '/kaggle/input/mcts-artifacts/nn-1dcnn_predict_uni95.pt'
# nn_1dcnn_artifacts_path = '/kaggle/input/mcts-artifacts/nn-1dcnn_predict_full.pt'

# Load the function from the file
nn_1dcnn_artifacts = torch.load(nn_1dcnn_artifacts_path, weights_only=False)

len(nn_1dcnn_artifacts['models'])

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.5.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


15

In [4]:
class SoftOrdering1DCNN(pl.LightningModule):

    def __init__(self, 
            num_input_dim: int,
            cat_input_dims: list[int],
            output_dim: int,
            sign_size: int = 32,
            cha_input: int = 16, 
            cha_hidden: int = 32,
            K: int = 2,
            dropout_input: float = 0.2,
            dropout_hidden: float = 0.2, 
            dropout_output: float = 0.2,
            embedding_dropout: float = 0.2,
            learning_rate: float = 1e-3,
            weight_decay: float = 1e-5,
            embedding_dim: Optional[List[int]] = None,
            pct_start: float = 0.2,
            div_factor: float = 10.0,
            final_div_factor: float = 1e4):
        super().__init__()
        self.save_hyperparameters()

        # Initialize embedding dimensions if not provided
        if embedding_dim is None:
            embedding_dim = [min(50, int(1 + np.ceil(np.sqrt(dim)))) for dim in cat_input_dims]
        elif len(embedding_dim) != len(cat_input_dims):
            raise ValueError("Length of embedding_dim must match number of categorical features.")
        
        self.embedding_dim = embedding_dim
        self.embedding_dropout = embedding_dropout
        
        # Create embedding layers
        self.embeddings = nn.ModuleList(
            [nn.Embedding(dim, emb_dim) for dim, emb_dim in zip(cat_input_dims, embedding_dim)]
        )
        self.embedding_dropout_layer = nn.Dropout(self.embedding_dropout)

        # Calculate total input dimension after embeddings
        total_embedding_dim = sum(self.embedding_dim)
        total_input_dim = num_input_dim + total_embedding_dim

        # CNN architecture parameters
        hidden_size = sign_size * cha_input
        self.sign_size1 = sign_size
        self.sign_size2 = sign_size//2
        self.output_size = (sign_size//4) * cha_hidden
        self.cha_input = cha_input
        self.cha_hidden = cha_hidden
        self.K = K

        # Input projection
        self.batch_norm1 = nn.BatchNorm1d(total_input_dim)
        self.dropout1 = nn.Dropout(dropout_input)
        dense1 = nn.Linear(total_input_dim, hidden_size, bias=False)
        self.dense1 = nn.utils.weight_norm(dense1)

        # 1st conv layer
        self.batch_norm_c1 = nn.BatchNorm1d(cha_input)
        conv1 = nn.Conv1d(
            cha_input, 
            cha_input*K, 
            kernel_size=5, 
            stride=1, 
            padding=2,  
            groups=cha_input, 
            bias=False)
        self.conv1 = nn.utils.weight_norm(conv1, dim=None)
        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size=self.sign_size2)

        # 2nd conv layer
        self.batch_norm_c2 = nn.BatchNorm1d(cha_input*K)
        self.dropout_c2 = nn.Dropout(dropout_hidden)
        conv2 = nn.Conv1d(
            cha_input*K, 
            cha_hidden, 
            kernel_size=3, 
            stride=1, 
            padding=1, 
            bias=False)
        self.conv2 = nn.utils.weight_norm(conv2, dim=None)

        # 3rd conv layer
        self.batch_norm_c3 = nn.BatchNorm1d(cha_hidden)
        self.dropout_c3 = nn.Dropout(dropout_hidden)
        conv3 = nn.Conv1d(
            cha_hidden, 
            cha_hidden, 
            kernel_size=3, 
            stride=1, 
            padding=1, 
            bias=False)
        self.conv3 = nn.utils.weight_norm(conv3, dim=None)

        # 4th conv layer
        self.batch_norm_c4 = nn.BatchNorm1d(cha_hidden)
        conv4 = nn.Conv1d(
            cha_hidden, 
            cha_hidden, 
            kernel_size=5, 
            stride=1, 
            padding=2, 
            groups=cha_hidden, 
            bias=False)
        self.conv4 = nn.utils.weight_norm(conv4, dim=None)

        self.avg_po_c4 = nn.AvgPool1d(kernel_size=4, stride=2, padding=1)
        self.flt = nn.Flatten()

        # Output head
        self.batch_norm2 = nn.BatchNorm1d(self.output_size)
        self.dropout2 = nn.Dropout(dropout_output)
        dense2 = nn.Linear(self.output_size, output_dim, bias=False)
        self.dense2 = nn.utils.weight_norm(dense2)

        # Training parameters
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.pct_start = pct_start
        self.div_factor = div_factor
        self.final_div_factor = final_div_factor

        # Initialize lists to store validation outputs
        self.validation_targets = []
        self.validation_predictions = []

    def forward(self, x_num, x_cat):
        # Process categorical variables
        embedded = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        embedded = torch.cat(embedded, dim=1)
        embedded = self.embedding_dropout_layer(embedded)
        
        # Concatenate numerical and embedded categorical features
        x = torch.cat([x_num, embedded], dim=1)

        # Input projection
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = nn.functional.celu(self.dense1(x))

        # Reshape for CNN
        x = x.reshape(x.shape[0], self.cha_input, self.sign_size1)

        # CNN backbone
        x = self.batch_norm_c1(x)
        x = nn.functional.leaky_relu(self.conv1(x))
        x = self.ave_po_c1(x)

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = nn.functional.leaky_relu(self.conv2(x))
        x_s = x

        x = self.batch_norm_c3(x)
        x = self.dropout_c3(x)
        x = nn.functional.leaky_relu(self.conv3(x))

        x = self.batch_norm_c4(x)
        x = self.conv4(x)
        x = x + x_s
        x = nn.functional.leaky_relu(x)

        x = self.avg_po_c4(x)
        x = self.flt(x)

        # Output head
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = self.dense2(x)
        x = nn.functional.hardtanh(x)

        return x.squeeze(-1)

    def training_step(self, batch, batch_idx):
        x_num, x_cat, y = batch
        y_hat = self(x_num, x_cat)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_num, x_cat, y = batch
        y_hat = self(x_num, x_cat)
        loss = F.mse_loss(y_hat, y)
        self.log('valid_loss', loss, prog_bar=True)
        # Store targets and predictions for later use
        self.validation_targets.append(y)
        self.validation_predictions.append(y_hat)
        return loss
    
    def predict_step(self, batch, batch_idx):
        if len(batch) == 2:
            x_num, x_cat = batch
        elif len(batch) == 3:
            x_num, x_cat, _ = batch
        y_hat = self(x_num, x_cat)
        return y_hat

    def on_validation_epoch_end(self):
        # Concatenate all targets and predictions
        y = torch.cat(self.validation_targets)
        y_hat = torch.cat(self.validation_predictions)
        rmse = torch.sqrt(F.mse_loss(y_hat, y))
        self.log('val_rmse', rmse, prog_bar=True)
        # Clear the lists for next epoch
        self.validation_targets.clear()
        self.validation_predictions.clear()
                
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(), 
            lr=self.learning_rate, 
            weight_decay=self.weight_decay,
        )
        scheduler = OneCycleLR(
            optimizer,
            max_lr=self.learning_rate,
            total_steps=self.trainer.estimated_stepping_batches,
            pct_start=self.pct_start,
            div_factor=self.div_factor,
            final_div_factor=self.final_div_factor,
            anneal_strategy='cos',
            cycle_momentum=True,
            base_momentum=0.85,
            max_momentum=0.95,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
            },
        }

In [5]:
class SoftOrdering1DCNNInference:
    def __init__(
        self,
        models_state_dicts,
        models_hparams,
        numerical_cols,
        categorical_cols,
        encoder,
        scaler,
        lgbm_encoders,
    ):
        """Initialize inference class with trained artifacts
        
        Args:
            models_state_dicts: List of model state dictionaries
            models_hparams: List of model hyperparameters
            numerical_cols: List of numerical column names
            categorical_cols: List of categorical column names
            encoder: Fitted OrdinalEncoder for categorical features
            scaler: Fitted StandardScaler for numerical features
            lgbm_encoders: List of LightGBM encoders for feature engineering
        """
        self.numerical_cols = numerical_cols
        self.categorical_cols = categorical_cols
        self.encoder = encoder
        self.scaler = scaler
        self.lgbm_encoders = lgbm_encoders

        # Load models
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.models = []
        for state_dict, hparams in zip(models_state_dicts, models_hparams):
            model = SoftOrdering1DCNN(**hparams)
            model.load_state_dict(state_dict)
            model.to(self.device)  # Move model to GPU
            model.eval()  # Set to evaluation mode
            self.models.append(model)

        print("len(numerical_cols):", len(numerical_cols))
        print("len(categorical_cols):", len(categorical_cols))

    def predict_array(self, df_test, batch_size=4096):
        """Make predictions on test data using DataLoader
        
        Args:
            df_test: pandas DataFrame containing test features
            batch_size: size of batches for inference
            
        Returns:
            numpy array of predictions
        """
        # Preprocess test data
        test_processed = process_test_data(
            df_test,
            self.numerical_cols,
            self.categorical_cols,
            self.encoder,
            self.scaler,
            include_position_features=False,
            include_text_features=False,
        )

        # Initialize predictions array
        predictions = np.zeros(len(df_test))

        # Get predictions from all models
        for lgbm_encoder, model in zip(self.lgbm_encoders, self.models):
            # Prepare numerical and categorical features
            X_test_num = test_processed[self.numerical_cols].copy()
            X_test_cat = test_processed[self.categorical_cols].copy()

            # Add LGBM encoder leaves features
            lgbm_features = lgbm_encoder.transform(
                test_processed[self.numerical_cols + self.categorical_cols]
            )
            X_test_cat = pd.concat([X_test_cat, lgbm_features], axis=1)
            _categorical_cols = self.categorical_cols + lgbm_encoder.new_columns

            # Create tensors
            X_num_tensor = torch.tensor(
                X_test_num[self.numerical_cols].values, 
                dtype=torch.float32,
                device=self.device
            )
            X_cat_tensor = torch.tensor(
                X_test_cat[_categorical_cols].values, 
                dtype=torch.int32,
                device=self.device
            )
            
            # Create TensorDataset and DataLoader
            dataset = torch.utils.data.TensorDataset(
                X_num_tensor, 
                X_cat_tensor
            )
            dataloader = torch.utils.data.DataLoader(
                dataset, 
                batch_size=batch_size,
                shuffle=False
            )
            
            # Process batches using DataLoader
            batch_predictions = []
            with torch.no_grad():
                for X_num_batch, X_cat_batch in dataloader:
                    pred_batch = model(X_num_batch, X_cat_batch).cpu()
                    batch_predictions.append(pred_batch)

            # Concatenate all batch predictions
            model_predictions = torch.cat(batch_predictions).numpy().flatten()
            predictions += model_predictions

        # Average predictions across models
        predictions /= len(self.models)
        return predictions

    def predict(self, test: polars.DataFrame, sample_sub: polars.DataFrame):
        test_pd = test.to_pandas().fillna(0)
        predictions = self.predict_array(test_pd)
        submission = sample_sub.with_columns(polars.Series("utility_agent1", predictions))
        return submission


# Create inference class
model_1dcnn = SoftOrdering1DCNNInference(
    models_state_dicts=nn_1dcnn_artifacts['models'],
    models_hparams=nn_1dcnn_artifacts['models_hparams'],
    numerical_cols=nn_1dcnn_artifacts['numerical_cols'],
    categorical_cols=nn_1dcnn_artifacts['categorical_cols'],
    encoder=nn_1dcnn_artifacts['encoder'],
    scaler=nn_1dcnn_artifacts['scaler'],
    lgbm_encoders=nn_1dcnn_artifacts['lgbm_encoders'],
)

/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


len(numerical_cols): 215
len(categorical_cols): 8


In [6]:
# sanity check #1
test = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv")
sample_sub = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv")
model_1dcnn.predict(test, sample_sub)

/tmp/ipykernel_23/2413545367.py:118: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_pd = test.to_pandas().fillna(0)


Id,utility_agent1
i64,f64
233234,0.136913
233235,-0.139139
233236,-0.040731


In [7]:
# %%time
# sanity check #2
# train = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv")
# test = train.drop(['num_wins_agent1', 'num_draws_agent1', 'num_losses_agent1', 'utility_agent1'])
# sample_sub = train.select(['Id', 'utility_agent1'])
# model_1dcnn.predict(test, sample_sub)

***
### MLP


In [8]:
# Specify the path where you want to save the serialized function

nn_mlp_artifacts_path = '/kaggle/input/mcts-artifacts/nn-mlp_predict_uni80.pt'
# nn_mlp_artifacts_path = '/kaggle/input/mcts-artifacts/nn-mlp_predict_uni95.pt'
# nn_mlp_artifacts_path = '/kaggle/input/mcts-artifacts/nn-mlp_predict_full.pt'

# Load the function from the file
nn_mlp_artifacts = torch.load(nn_mlp_artifacts_path, weights_only=False)

len(nn_mlp_artifacts['models'])

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.5.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


15

In [9]:
class MLP(pl.LightningModule):

    def __init__(self, 
            num_input_dim: int,
            cat_input_dims: list[int],
            output_dim: int,
            layers: str,
            dropout: float,
            embedding_dropout: float,
            learning_rate: float = 1e-3,
            weight_decay: float = 1e-5,
            initialization: str = 'kaiming_uniform',
            embedding_dim: Optional[List[int]] = None,
            pct_start: float = 0.2,
            div_factor: float = 10.0,
            final_div_factor: float = 1e4,
        ):
        super().__init__()
        self.save_hyperparameters()
        self.dropout = dropout
        self.embedding_dropout = embedding_dropout
        self.pct_start = pct_start
        self.div_factor = div_factor
        self.final_div_factor = final_div_factor

        # Initialize embedding dimensions if not provided
        if embedding_dim is None:
            # Rule of thumb: min(50, num_unique // 2 + 1) for each categorical feature
            embedding_dim = [min(50, int(1 + np.ceil(np.sqrt(dim)))) for dim in cat_input_dims]

        elif len(embedding_dim) != len(cat_input_dims):
            raise ValueError("Length of embedding_dim must match number of categorical features.")

        self.embedding_dim = embedding_dim

        # Create embedding layers
        self.create_embeddings(cat_input_dims, embedding_dim)

        # Create backbone layers
        self.create_backbone(num_input_dim, layers)

        # Create head layers
        self.create_head(output_dim)

        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.initialization = initialization

        self._init_weights()

        # Initialize lists to store validation outputs
        self.validation_targets = []
        self.validation_predictions = []

    def create_embeddings(self, cat_input_dims: list[int], embedding_dim: list[int]):
        self.embeddings = nn.ModuleList(
            [nn.Embedding(dim, emb_dim) for dim, emb_dim in zip(cat_input_dims, embedding_dim)]
        )
        self.embedding_dropout_layer = nn.Dropout(self.embedding_dropout)

    def create_backbone(self, num_input_dim: int, layers: str):
        # Calculate total input dimension after embeddings
        total_embedding_dim = sum(self.embedding_dim)
        total_input_dim = num_input_dim + total_embedding_dim

        # Parse layers string
        layer_sizes = [int(size) for size in layers.split('-')]

        # Create backbone network layers
        backbone_layers = []
        prev_size = total_input_dim
        for size in layer_sizes:
            backbone_layers.extend([
                nn.BatchNorm1d(prev_size),
                nn.Linear(prev_size, size),
                nn.ReLU(),
                nn.Dropout(self.hparams.dropout),
            ])
            prev_size = size
        self.backbone = nn.Sequential(*backbone_layers)
        self.backbone_output_size = prev_size

    def create_head(self, output_dim: int):
        # Output layer
        self.head = nn.Sequential(
            nn.BatchNorm1d(self.backbone_output_size),
            nn.Linear(self.backbone_output_size, output_dim)
        )

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                if any(module is m for m in self.head.modules()):
                    nn.init.xavier_uniform_(module.weight, gain=nn.init.calculate_gain('tanh'))
                else:
                    if self.initialization == 'kaiming_uniform':
                        nn.init.kaiming_uniform_(module.weight, nonlinearity='relu')
                    elif self.initialization == 'kaiming_normal':
                        nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
                    elif self.initialization == 'xavier_uniform':
                        nn.init.xavier_uniform_(module.weight, gain=nn.init.calculate_gain('relu'))
                    elif self.initialization == 'xavier_normal':
                        nn.init.xavier_normal_(module.weight, gain=nn.init.calculate_gain('relu'))
                    else:
                        raise ValueError(f"Unsupported initialization method: {self.initialization}")
                
                # Initialize bias to small values
                if module.bias is not None:
                    nn.init.uniform_(module.bias, -0.1, 0.1)

    def forward(self, x_num, x_cat):
        # Process categorical variables
        embedded = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        embedded = torch.cat(embedded, dim=1)
        embedded = self.embedding_dropout_layer(embedded)
        
        # Concatenate numerical and embedded categorical features
        x = torch.cat([x_num, embedded], dim=1)
        
        # Pass through backbone
        x = self.backbone(x)
        
        # Pass through head
        x = self.head(x)
        x = nn.functional.hardtanh(x)

        return x.squeeze(-1)

    def training_step(self, batch, batch_idx):
        x_num, x_cat, y = batch
        y_hat = self(x_num, x_cat)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_num, x_cat, y = batch
        y_hat = self(x_num, x_cat)
        loss = F.mse_loss(y_hat, y)
        self.log('valid_loss', loss, prog_bar=True)
        # Store targets and predictions for later use
        self.validation_targets.append(y)
        self.validation_predictions.append(y_hat)
        return loss
    
    def predict_step(self, batch, batch_idx):
        if len(batch) == 2:
            x_num, x_cat = batch
        elif len(batch) == 3:
            x_num, x_cat, _ = batch
        y_hat = self(x_num, x_cat)
        return y_hat

    def on_validation_epoch_end(self):
        # Concatenate all targets and predictions
        y = torch.cat(self.validation_targets)
        y_hat = torch.cat(self.validation_predictions)
        rmse = torch.sqrt(F.mse_loss(y_hat, y))
        self.log('val_rmse', rmse, prog_bar=True)
        # Clear the lists for next epoch
        self.validation_targets.clear()
        self.validation_predictions.clear()
                
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(), 
            lr=self.learning_rate, 
            weight_decay=self.weight_decay,
        )
        scheduler = OneCycleLR(
            optimizer,
            max_lr=self.learning_rate,
            total_steps=self.trainer.estimated_stepping_batches,
            pct_start=self.pct_start,
            div_factor=self.div_factor,
            final_div_factor=self.final_div_factor,
            anneal_strategy='cos',
            cycle_momentum=True,
            base_momentum=0.85,
            max_momentum=0.95,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
            },
        }

In [10]:
class MLPInference:
    def __init__(
        self,
        models_state_dicts,
        models_hparams,
        numerical_cols,
        categorical_cols,
        encoder,
        scaler,
        lgbm_encoders,
    ):
        """Initialize inference class with trained artifacts
        
        Args:
            models_state_dicts: List of model state dictionaries
            models_hparams: List of model hyperparameters
            numerical_cols: List of numerical column names
            categorical_cols: List of categorical column names
            encoder: Fitted OrdinalEncoder for categorical features
            scaler: Fitted StandardScaler for numerical features
            lgbm_encoders: List of LightGBM encoders for feature engineering
        """
        self.numerical_cols = numerical_cols
        self.categorical_cols = categorical_cols
        self.encoder = encoder
        self.scaler = scaler
        self.lgbm_encoders = lgbm_encoders

        # Load models
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.models = []
        for state_dict, hparams in zip(models_state_dicts, models_hparams):
            model = MLP(**hparams)
            model.load_state_dict(state_dict)
            model.to(self.device)  # Move model to GPU
            model.eval()  # Set to evaluation mode
            self.models.append(model)

        print("len(numerical_cols):", len(numerical_cols))
        print("len(categorical_cols):", len(categorical_cols))

    def predict_array(self, df_test, batch_size=4096):
        """Make predictions on test data using DataLoader
        
        Args:
            df_test: pandas DataFrame containing test features
            batch_size: size of batches for inference
            
        Returns:
            numpy array of predictions
        """
        # Preprocess test data
        test_processed = process_test_data(
            df_test,
            self.numerical_cols,
            self.categorical_cols,
            self.encoder,
            self.scaler,
            include_position_features=False,
            include_text_features=False,
        )

        # Initialize predictions array
        predictions = np.zeros(len(df_test))

        # Get predictions from all models
        for lgbm_encoder, model in zip(self.lgbm_encoders, self.models):
            # Prepare numerical and categorical features
            X_test_num = test_processed[self.numerical_cols].copy()
            X_test_cat = test_processed[self.categorical_cols].copy()

            # Add LGBM encoder leaves features
            lgbm_features = lgbm_encoder.transform(
                test_processed[self.numerical_cols + self.categorical_cols]
            )
            X_test_cat = pd.concat([X_test_cat, lgbm_features], axis=1)
            _categorical_cols = self.categorical_cols + lgbm_encoder.new_columns

            # Create tensors
            X_num_tensor = torch.tensor(
                X_test_num[self.numerical_cols].values, 
                dtype=torch.float32,
                device=self.device
            )
            X_cat_tensor = torch.tensor(
                X_test_cat[_categorical_cols].values, 
                dtype=torch.int32,
                device=self.device,
            )
            
            # Create TensorDataset and DataLoader
            dataset = torch.utils.data.TensorDataset(
                X_num_tensor, 
                X_cat_tensor
            )
            dataloader = torch.utils.data.DataLoader(
                dataset, 
                batch_size=batch_size,
                shuffle=False
            )
            
            # Process batches using DataLoader
            batch_predictions = []
            with torch.no_grad():
                for X_num_batch, X_cat_batch in dataloader:
                    pred_batch = model(X_num_batch, X_cat_batch).cpu()
                    batch_predictions.append(pred_batch)

            # Concatenate all batch predictions
            model_predictions = torch.cat(batch_predictions).numpy().flatten()
            predictions += model_predictions

        # Average predictions across models
        predictions /= len(self.models)
        return predictions

    def predict(self, test: polars.DataFrame, sample_sub: polars.DataFrame):
        test_pd = test.to_pandas().fillna(0)
        predictions = self.predict_array(test_pd)
        submission = sample_sub.with_columns(polars.Series("utility_agent1", predictions))
        return submission


# Create inference class
model_mlp = MLPInference(
    models_state_dicts=nn_mlp_artifacts['models'],
    models_hparams=nn_mlp_artifacts['models_hparams'],
    numerical_cols=nn_mlp_artifacts['numerical_cols'],
    categorical_cols=nn_mlp_artifacts['categorical_cols'],
    encoder=nn_mlp_artifacts['encoder'],
    scaler=nn_mlp_artifacts['scaler'],
    lgbm_encoders=nn_mlp_artifacts['lgbm_encoders'],
)

len(numerical_cols): 215
len(categorical_cols): 8


In [11]:
# sanity check #1
test = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv")
sample_sub = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv")
model_mlp.predict(test, sample_sub)

/tmp/ipykernel_23/1309123747.py:118: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_pd = test.to_pandas().fillna(0)


Id,utility_agent1
i64,f64
233234,0.109035
233235,-0.138016
233236,-0.013962


In [12]:
# %%time
# sanity check #2
# train = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv")
# test = train.drop(['num_wins_agent1', 'num_draws_agent1', 'num_losses_agent1', 'utility_agent1'])
# sample_sub = train.select(['Id', 'utility_agent1'])
# model_mlp.predict(test, sample_sub)

***
### gandalf

In [13]:
# Specify the path where you want to save the serialized function
nn_gandalf_artifacts_path = '/kaggle/input/mcts-artifacts/nn-gandalf_predict_uni80.pt'
# nn_gandalf_artifacts_path = '/kaggle/input/mcts-artifacts/nn-gandalf_predict_uni95.pt'
# nn_gandalf_artifacts_path = '/kaggle/input/mcts-artifacts/nn-gandalf_predict_full.pt'

# Load the function from the file
nn_gandalf_artifacts = torch.load(nn_gandalf_artifacts_path, weights_only=False)

len(nn_gandalf_artifacts['models'])

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.5.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


15

In [14]:
class GANDALF(pl.LightningModule):

    def __init__(self, 
            num_input_dim: int,
            cat_input_dims: list[int],
            output_dim: int,
            dropout: float,
            embedding_dropout: float,
            learning_rate: float = 1e-3,
            weight_decay: float = 1e-5,
            initialization: str = 'kaiming_uniform',
            embedding_dim: Optional[List[int]] = None,
            pct_start: float = 0.2,
            div_factor: float = 10.0,
            final_div_factor: float = 1e4,
            n_stages: int = 6,
            feature_mask_function: Callable = t_softmax,
            feature_sparsity: float = 0.3,
            learnable_sparsity: bool = True
        ):
        super().__init__()
        self.save_hyperparameters()
        self.dropout = dropout
        self.embedding_dropout = embedding_dropout
        self.pct_start = pct_start
        self.div_factor = div_factor
        self.final_div_factor = final_div_factor

        # Initialize embedding dimensions if not provided
        if embedding_dim is None:
            # Rule of thumb: min(50, num_unique // 2 + 1) for each categorical feature
            embedding_dim = [min(50, int(1 + np.ceil(np.sqrt(dim)))) for dim in cat_input_dims]

        elif len(embedding_dim) != len(cat_input_dims):
            raise ValueError("Length of embedding_dim must match number of categorical features.")

        self.embedding_dim = embedding_dim

        # Create embedding layers
        self.create_embeddings(cat_input_dims, embedding_dim)

        # Create backbone layers
        self.create_backbone(
            num_input_dim,
            n_stages,
            feature_mask_function,
            dropout,
            feature_sparsity,
            learnable_sparsity
        )

        # Create head layers
        self.create_head(output_dim)

        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.initialization = initialization

        self._init_weights()

        # Initialize lists to store validation outputs
        self.validation_targets = []
        self.validation_predictions = []

    def create_embeddings(self, cat_input_dims: list[int], embedding_dim: list[int]):
        self.embeddings = nn.ModuleList(
            [nn.Embedding(dim, emb_dim) for dim, emb_dim in zip(cat_input_dims, embedding_dim)]
        )
        self.embedding_dropout_layer = nn.Dropout(self.embedding_dropout)

    def create_backbone(
            self, 
            num_input_dim: int,
            n_stages: int,
            feature_mask_function: Callable,
            dropout: float,
            feature_sparsity: float,
            learnable_sparsity: bool
        ):
        # Calculate total input dimension after embeddings
        total_embedding_dim = sum(self.embedding_dim)
        total_input_dim = num_input_dim + total_embedding_dim

        self.backbone = GatedFeatureLearningUnit(
            n_features_in=total_input_dim,
            n_stages=n_stages,
            feature_mask_function=feature_mask_function,
            dropout=dropout,
            feature_sparsity=feature_sparsity,
            learnable_sparsity=learnable_sparsity,
        )
        self.backbone_output_size = total_input_dim

    def create_head(self, output_dim: int):
        # Output layer
        self.head = nn.Sequential(
            nn.BatchNorm1d(self.backbone_output_size),
            nn.Linear(self.backbone_output_size, output_dim)
        )

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                if any(module is m for m in self.head.modules()):
                    nn.init.xavier_uniform_(module.weight, gain=nn.init.calculate_gain('tanh'))
                else:
                    if self.initialization == 'kaiming_uniform':
                        nn.init.kaiming_uniform_(module.weight, nonlinearity='relu')
                    elif self.initialization == 'kaiming_normal':
                        nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
                    elif self.initialization == 'xavier_uniform':
                        nn.init.xavier_uniform_(module.weight, gain=nn.init.calculate_gain('relu'))
                    elif self.initialization == 'xavier_normal':
                        nn.init.xavier_normal_(module.weight, gain=nn.init.calculate_gain('relu'))
                    else:
                        raise ValueError(f"Unsupported initialization method: {self.initialization}")
                
                # Initialize bias to small values
                if module.bias is not None:
                    nn.init.uniform_(module.bias, -0.1, 0.1)

    def forward(self, x_num, x_cat):
        # Process categorical variables
        embedded = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        embedded = torch.cat(embedded, dim=1)
        embedded = self.embedding_dropout_layer(embedded)
        
        # Concatenate numerical and embedded categorical features
        x = torch.cat([x_num, embedded], dim=1)
        
        # Pass through backbone
        x = self.backbone(x)
        
        # Pass through head
        x = self.head(x)
        x = nn.functional.hardtanh(x)

        return x.squeeze(-1)

    def training_step(self, batch, batch_idx):
        x_num, x_cat, y = batch
        y_hat = self(x_num, x_cat)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_num, x_cat, y = batch
        y_hat = self(x_num, x_cat)
        loss = F.mse_loss(y_hat, y)
        self.log('valid_loss', loss, prog_bar=True)
        # Store targets and predictions for later use
        self.validation_targets.append(y)
        self.validation_predictions.append(y_hat)
        return loss
    
    def predict_step(self, batch, batch_idx):
        if len(batch) == 2:
            x_num, x_cat = batch
        elif len(batch) == 3:
            x_num, x_cat, _ = batch
        y_hat = self(x_num, x_cat)
        return y_hat

    def on_validation_epoch_end(self):
        # Concatenate all targets and predictions
        y = torch.cat(self.validation_targets)
        y_hat = torch.cat(self.validation_predictions)
        rmse = torch.sqrt(F.mse_loss(y_hat, y))
        self.log('val_rmse', rmse, prog_bar=True)
        # Clear the lists for next epoch
        self.validation_targets.clear()
        self.validation_predictions.clear()
                
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(), 
            lr=self.learning_rate, 
            weight_decay=self.weight_decay,
        )
        scheduler = OneCycleLR(
            optimizer,
            max_lr=self.learning_rate,
            total_steps=self.trainer.estimated_stepping_batches,
            pct_start=self.pct_start,
            div_factor=self.div_factor,
            final_div_factor=self.final_div_factor,
            anneal_strategy='cos',
            cycle_momentum=True,
            base_momentum=0.85,
            max_momentum=0.95,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
            },
        }

In [15]:
class GandalfInference:
    def __init__(
        self,
        models_state_dicts,
        models_hparams,
        numerical_cols,
        categorical_cols,
        encoder,
        scaler,
        lgbm_encoders,
    ):
        """Initialize inference class with trained artifacts
        
        Args:
            models_state_dicts: List of model state dictionaries
            models_hparams: List of model hyperparameters
            numerical_cols: List of numerical column names
            categorical_cols: List of categorical column names
            encoder: Fitted OrdinalEncoder for categorical features
            scaler: Fitted StandardScaler for numerical features
            lgbm_encoders: List of LightGBM encoders for feature engineering
        """
        self.numerical_cols = numerical_cols
        self.categorical_cols = categorical_cols
        self.encoder = encoder
        self.scaler = scaler
        self.lgbm_encoders = lgbm_encoders

        # Load models
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.models = []
        for state_dict, hparams in zip(models_state_dicts, models_hparams):
            model = GANDALF(**hparams)
            model.load_state_dict(state_dict)
            model.to(self.device)  # Move model to GPU
            model.eval()  # Set to evaluation mode
            self.models.append(model)

        print(f"Using device: {self.device}")
        print("len(numerical_cols):", len(numerical_cols))
        print("len(categorical_cols):", len(categorical_cols))

    def predict_array(self, df_test, batch_size=4096):
        """Make predictions on test data using DataLoader
        
        Args:
            df_test: pandas DataFrame containing test features
            batch_size: size of batches for inference
            
        Returns:
            numpy array of predictions
        """
        # Preprocess test data
        test_processed = process_test_data(
            df_test,
            self.numerical_cols,
            self.categorical_cols,
            self.encoder,
            self.scaler,
            include_position_features=False,
            include_text_features=False,
        )

        # Initialize predictions array
        predictions = np.zeros(len(df_test))

        # Get predictions from all models
        for lgbm_encoder, model in zip(self.lgbm_encoders, self.models):
            # Prepare numerical and categorical features
            X_test_num = test_processed[self.numerical_cols].copy()
            X_test_cat = test_processed[self.categorical_cols].copy()

            # Add LGBM encoder leaves features
            lgbm_features = lgbm_encoder.transform(
                test_processed[self.numerical_cols + self.categorical_cols]
            )
            X_test_cat = pd.concat([X_test_cat, lgbm_features], axis=1)
            _categorical_cols = self.categorical_cols + lgbm_encoder.new_columns

            # Create tensors
            X_num_tensor = torch.tensor(
                X_test_num[self.numerical_cols].values, 
                dtype=torch.float32,
                device=self.device
            )
            X_cat_tensor = torch.tensor(
                X_test_cat[_categorical_cols].values, 
                dtype=torch.int32,
                device=self.device
            )
            
            # Create TensorDataset and DataLoader
            dataset = torch.utils.data.TensorDataset(
                X_num_tensor, 
                X_cat_tensor
            )
            dataloader = torch.utils.data.DataLoader(
                dataset, 
                batch_size=batch_size,
                shuffle=False
            )
            
            # Process batches using DataLoader
            batch_predictions = []
            with torch.no_grad():
                for X_num_batch, X_cat_batch in dataloader:
                    pred_batch = model(X_num_batch, X_cat_batch).cpu()
                    batch_predictions.append(pred_batch)

            # Concatenate all batch predictions
            model_predictions = torch.cat(batch_predictions).numpy().flatten()
            predictions += model_predictions

        # Average predictions across models
        predictions /= len(self.models)
        return predictions

    def predict(self, test: polars.DataFrame, sample_sub: polars.DataFrame):
        test_pd = test.to_pandas().fillna(0)
        predictions = self.predict_array(test_pd)
        submission = sample_sub.with_columns(polars.Series("utility_agent1", predictions))
        return submission


# Create inference class
model_gandalf = GandalfInference(
    models_state_dicts=nn_gandalf_artifacts['models'],
    models_hparams=nn_gandalf_artifacts['models_hparams'],
    numerical_cols=nn_gandalf_artifacts['numerical_cols'],
    categorical_cols=nn_gandalf_artifacts['categorical_cols'],
    encoder=nn_gandalf_artifacts['encoder'],
    scaler=nn_gandalf_artifacts['scaler'],
    lgbm_encoders=nn_gandalf_artifacts['lgbm_encoders'],
)

Using device: cuda
len(numerical_cols): 215
len(categorical_cols): 8


In [16]:
# sanity check #1
test = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv")
sample_sub = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv")
model_gandalf.predict(test, sample_sub)

/tmp/ipykernel_23/1597166422.py:119: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_pd = test.to_pandas().fillna(0)


Id,utility_agent1
i64,f64
233234,0.16078
233235,-0.166201
233236,-0.03351


In [17]:
# %%time
# sanity check #2
# train = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv")
# test = train.drop(['num_wins_agent1', 'num_draws_agent1', 'num_losses_agent1', 'utility_agent1'])
# sample_sub = train.select(['Id', 'utility_agent1'])
# model_gandalf.predict(test, sample_sub)

***
### catboost

In [18]:
# Specify the path where you want to save the serialized function

catboost_artifacts_path = '/kaggle/input/mcts-artifacts/catboost_text_predict_int99.pkl'
# catboost_artifacts_path = '/kaggle/input/mcts-artifacts/catboost_predict_int99.pkl'


# Load the function from the file
with open(catboost_artifacts_path, 'rb') as f:
    catboost_artifacts = pickle.load(f)

len(catboost_artifacts["models"])

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.5.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


15

In [19]:
class CATBoostInference:
    def __init__(
        self,
        models,
        numerical_cols,
        categorical_cols,
        encoder,
        scaler,
        text_cols=None,
    ):
        """Initialize inference class with trained artifacts
        
        Args:
            models: List of trained CatBoost models
            numerical_cols: List of numerical column names
            categorical_cols: List of categorical column names
            encoder: Fitted OrdinalEncoder for categorical features
            scaler: Fitted StandardScaler for numerical features (optional)
            text_cols: List of text columns (optional)
        """
        self.models = models
        self.numerical_cols = numerical_cols
        self.categorical_cols = categorical_cols
        self.text_cols = text_cols
        self.encoder = encoder
        self.scaler = scaler

        print("len(numerical_cols):", len(numerical_cols))
        print("len(categorical_cols):", len(categorical_cols))
        
    def predict_array(self, df_test):
        """Make predictions on test data
        
        Args:
            df_test: pandas DataFrame containing test features
            
        Returns:
            numpy array of predictions
        """
        # Preprocess test data
        test_processed = process_test_data(
            df_test,
            self.numerical_cols,
            self.categorical_cols,
            self.encoder,
            self.scaler,
            include_position_features=True,
            include_text_features=True,
        )
        
        # Create CatBoost Pool for test data
        features = self.numerical_cols + self.categorical_cols
        pool_kwargs = {
            'data': test_processed[features],
            'cat_features': self.categorical_cols,
        }
        
        if self.text_cols is not None:
            features += self.text_cols
            pool_kwargs['data'] = test_processed[features]
            pool_kwargs['text_features'] = self.text_cols
            
        test_pool = cb.Pool(**pool_kwargs)
        
        # Get predictions from all models
        predictions = np.mean([
            model.predict(test_pool)
            for model in self.models
        ], axis=0)
        # predictions = np.clip(predictions, -1, 1)
        
        return predictions
    

    def predict(self, test: polars.DataFrame, sample_sub: polars.DataFrame):
        test_pd = test.to_pandas()
        predictions = self.predict_array(test_pd)
        submission = sample_sub.with_columns(polars.Series("utility_agent1", predictions))
        return submission


model_catboost = CATBoostInference(
    # models=catboost_artifacts["models"][:5],
    # models=catboost_artifacts["models"][5:10],
    # models=catboost_artifacts["models"][10:],
    models=catboost_artifacts["models"],
    numerical_cols=catboost_artifacts["numerical_cols"],
    categorical_cols=catboost_artifacts["categorical_cols"],
    text_cols=catboost_artifacts["text_cols"],
    encoder=catboost_artifacts["encoder"],
    scaler=catboost_artifacts["scaler"],
)

len(numerical_cols): 362
len(categorical_cols): 8


In [20]:
# sanity check
test = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv")
sample_sub = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv")
model_catboost.predict(test, sample_sub)

Id,utility_agent1
i64,f64
233234,0.154499
233235,-0.18696
233236,0.015267


In [21]:
# %%time
# sanity check #2
# train = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv")
# test = train.drop(['num_wins_agent1', 'num_draws_agent1', 'num_losses_agent1', 'utility_agent1'])
# sample_sub = train.select(['Id', 'utility_agent1'])
# model_catboost.predict(test, sample_sub)

***
### lightgbm

In [22]:
# Specify the path where you want to save the serialized function
# lightgbm_artifacts_path = '/kaggle/input/mcts-artifacts/lightgbm_predict_uni80.pkl'
# lightgbm_artifacts_path = '/kaggle/input/mcts-artifacts/lightgbm_linear_predict_uni95.pkl'
# lightgbm_artifacts_path = '/kaggle/input/mcts-artifacts/lightgbm_predict_fsv24.pkl'

# Load the function from the file
# with open(lightgbm_artifacts_path, 'rb') as f:
#     lightgbm_artifacts = pickle.load(f)
# len(lightgbm_artifacts["models"])

In [23]:
class LightGBMInference:
    def __init__(
        self,
        models,
        numerical_cols,
        categorical_cols,
        encoder,
        scaler,
    ):
        """Initialize inference class with trained artifacts
        
        Args:
            models: List of trained LightGBM models
            numerical_cols: List of numerical column names
            categorical_cols: List of categorical column names
            encoder: Fitted OrdinalEncoder for categorical features
            scaler: Fitted StandardScaler for numerical features (optional)
        """
        self.models = models
        self.numerical_cols = numerical_cols
        self.categorical_cols = categorical_cols
        self.encoder = encoder
        self.scaler = scaler

        print("len(numerical_cols):", len(numerical_cols))
        print("len(categorical_cols):", len(categorical_cols))
        
    def predict_array(self, df_test):
        """Make predictions on test data
        
        Args:
            df_test: pandas DataFrame containing test features
            
        Returns:
            numpy array of predictions
        """
        # Preprocess test data
        test_processed = process_test_data(
            df_test,
            self.numerical_cols,
            self.categorical_cols,
            self.encoder,
            self.scaler,
            include_position_features=True,
            include_text_features=True,
        )
        
        # Get predictions from all models
        predictions = np.mean([
            model.predict(test_processed[self.numerical_cols + self.categorical_cols])
            for model in self.models
        ], axis=0)
        # predictions = np.clip(predictions, -1, 1)
        
        return predictions

    def predict(self, test: polars.DataFrame, sample_sub: polars.DataFrame):
        test_pd = test.to_pandas()
        predictions = self.predict_array(test_pd)
        submission = sample_sub.with_columns(polars.Series("utility_agent1", predictions))
        return submission

# model_lgbm = LightGBMInference(**lightgbm_artifacts)

In [24]:
# sanity check
# test = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv")
# sample_sub = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv")
# model_lgbm.predict(test, sample_sub)

***
### xgboost

In [25]:
# Specify the path where you want to save the serialized function
# xgboost_artifacts_path = '/kaggle/input/mcts-artifacts/xgboost_predict_uni90.pkl'
# xgboost_artifacts_path = '/kaggle/input/mcts-artifacts/xgboost_predict_fsv24.pkl'

# Load the function from the file
# with open(xgboost_artifacts_path, 'rb') as f:
#     xgboost_artifacts = pickle.load(f)
# len(xgboost_artifacts["models"])

In [26]:
class XGBoostInference:
    def __init__(
        self,
        models,
        numerical_cols,
        categorical_cols,
        encoder,
        scaler,
    ):
        """Initialize inference class with trained artifacts
        
        Args:
            models: List of trained XGBoost models
            numerical_cols: List of numerical column names
            categorical_cols: List of categorical column names
            encoder: Fitted OrdinalEncoder for categorical features
            scaler: Fitted StandardScaler for numerical features
        """
        self.models = models
        self.numerical_cols = numerical_cols
        self.categorical_cols = categorical_cols
        self.encoder = encoder
        self.scaler = scaler

        print("len(numerical_cols):", len(numerical_cols))
        print("len(categorical_cols):", len(categorical_cols))
        
    def predict_array(self, df_test):
        """Make predictions on test data
        
        Args:
            df_test: pandas DataFrame containing test features
            
        Returns:
            numpy array of predictions
        """
        # Preprocess test data
        test_processed = process_test_data(
            df_test,
            self.numerical_cols,
            self.categorical_cols,
            self.encoder,
            self.scaler,
            include_position_features=True,
            include_text_features=True,
        )
        
        # Create feature types list for XGBoost
        feature_types = [
            "c" if col in self.categorical_cols else "q" 
            for col in self.numerical_cols + self.categorical_cols
        ]
        
        # Create XGBoost DMatrix for test data
        test_dmatrix = xgb.DMatrix(
            data=test_processed[self.numerical_cols + self.categorical_cols],
            feature_types=feature_types,
            enable_categorical=True
        )
        
        # Get predictions from all models
        predictions = np.mean([
            model.predict(test_dmatrix)
            for model in self.models
        ], axis=0)
        # predictions = np.clip(predictions, -1, 1)
        
        return predictions
    
    def predict(self, test: polars.DataFrame, sample_sub: polars.DataFrame):
        test_pd = test.to_pandas()
        predictions = self.predict_array(test_pd)
        submission = sample_sub.with_columns(polars.Series("utility_agent1", predictions))
        return submission


# model_xgboost = XGBoostInference(
#    models=xgboost_artifacts["models"],
#    numerical_cols=xgboost_artifacts["numerical_cols"],
#    categorical_cols=xgboost_artifacts["categorical_cols"],
#    encoder=xgboost_artifacts["encoder"],
#    scaler=xgboost_artifacts["scaler"],
#)

In [27]:
# sanity check
# test = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv")
# sample_sub = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv")
# model_xgboost.predict(test, sample_sub)

***
### blend

In [28]:
def predict(test, sample_sub):
    pred_cat = model_catboost.predict(test, sample_sub)
    pred_1dcnn = model_1dcnn.predict(test, sample_sub)
    pred_mlp = model_mlp.predict(test, sample_sub)
    pred_gandalf = model_gandalf.predict(test, sample_sub)

    out = pred_cat.clone()
    out = out.with_columns(
        (
            0.4299 * pred_cat["utility_agent1"] +
            0.1265 * pred_mlp["utility_agent1"] + 
            0.2767 * pred_1dcnn["utility_agent1"] +
            0.1669 * pred_gandalf["utility_agent1"]
        ).clip(-1, 1).alias("utility_agent1")
    )

    return out

In [29]:
# sanity check
test = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv")
sample_sub = polars.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv")
predict(test, sample_sub)

/tmp/ipykernel_23/2413545367.py:118: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_pd = test.to_pandas().fillna(0)
/tmp/ipykernel_23/1309123747.py:118: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_pd = test.to_pandas().fillna(0)
/tmp/ipykernel_23/1597166422.py:119: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_pd = test.to

Id,utility_agent1
i64,f64
233234,0.14493
233235,-0.164072
233236,-0.012066


***
### inference

In [30]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )

/tmp/ipykernel_23/2413545367.py:118: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_pd = test.to_pandas().fillna(0)
/tmp/ipykernel_23/1309123747.py:118: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_pd = test.to_pandas().fillna(0)
/tmp/ipykernel_23/1597166422.py:119: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_pd = test.to

***